In [12]:
# Import the libraries
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import tempfile 
import os
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import splitfolders
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
import cv2
import shutil
from keras.applications.vgg16 import decode_predictions
from keras import losses

In [8]:
train=pd.read_csv(r"C:\Users\Dell\Documents\projects\image search engine\trainval.csv")

### loading the training dataset

In [2]:
test.columns

NameError: name 'test' is not defined

In [10]:
test=pd.read_csv(r"C:\Users\Dell\Documents\projects\image search engine\test.csv")

### loading the test dataset

In [11]:
train.rename(columns={'Abyssinian_100':'name','1':'id','1.1':'species','1.2':'breed'},inplace=True)

In [12]:
test.rename(columns={'Abyssinian_201':'name','1':'id','1.1':'species','1.2':'breed'},inplace=True)

In [13]:
new=pd.concat([train,test])
new=new.sort_values(by='name')
new

,name,id,species,breed
1894,Abyssinian_1,1,1,1
9,Abyssinian_10,1,1,1
0,Abyssinian_101,1,1,1
1,Abyssinian_102,1,1,1
2,Abyssinian_103,1,1,1
...,...,...,...,...
3662,yorkshire_terrier_95,37,2,25
3663,yorkshire_terrier_96,37,2,25
3664,yorkshire_terrier_97,37,2,25
3665,yorkshire_terrier_98,37,2,25


In [14]:
y=new['breed'].copy()

In [15]:
x_train, x_test, y_train, y_test = train_test_split(new, y, test_size=0.2, random_state=42)

In [16]:
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

In [17]:
datagen = ImageDataGenerator(width_shift_range=0.2,          
                    height_shift_range=0.2, 
                    rotation_range=120, 
                    horizontal_flip=True,
                    vertical_flip=True,
                    zoom_range = 0.2,
                    shear_range= 0.2)
val_datagen = ImageDataGenerator() 
test_datagen = ImageDataGenerator()

### Used Imagedatagenerator for Image augmentation

In [18]:
train_generator = datagen.flow_from_directory(
    "C:\\Users\\Dell\\Documents\\projects\\image search engine\\oxford-iiit-pet\\images.tar\\images\\train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    "C:\\Users\\Dell\\Documents\\projects\\image search engine\\oxford-iiit-pet\\images.tar\\images\\val",
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)  

# Test the model on a few samples
test_generator = val_datagen.flow_from_directory(
    "C:\\Users\\Dell\\Documents\\projects\\image search engine\\oxford-iiit-pet\\images.tar\\images\\test",
    target_size=(224,224),
    batch_size=1,
    class_mode='categorical', 
    shuffle=False
)

Found 5875 images belonging to 37 classes.
Found 735 images belonging to 37 classes.
Found 733 images belonging to 37 classes.


In [34]:
type(train_generator)

keras.src.preprocessing.image.DirectoryIterator

In [49]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  
x = keras.layers.Dense(180,activation='relu')(x)
x = keras.layers.Dropout(0.2)(x) 
x = keras.layers.Dense(120,activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(60,activation='relu')(x)
x = keras.layers.Dropout(0.2)(x) 
outputs = keras.layers.Dense(37,activation='softmax')(x)
model = keras.Model(inputs, outputs)

model.summary(show_trainable=True)

Model: "model_5"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_16 (InputLayer)       [(None, 224, 224, 3)]     0         Y          
                                                                            
 vgg16 (Functional)          (None, 7, 7, 512)         1471468   N          
                                                       8                    
                                                                            
 global_average_pooling2d_8  (None, 512)               0         Y          
  (GlobalAveragePooling2D)                                                  
                                                                            
 dropout_11 (Dropout)        (None, 512)               0         Y          
                                                                            
 dense_11 (Dense)            (None, 180)               9234

#### 1. We'll have vgg16 with imagenet weights as the base layer without including the top layer. We'll set the training as 'False' at first so that we wont affect its weights. 
#### 2. Then we'll add multiple sets of dense, drop-out, pooling layers to this with training parameter set as true. These will have activation function as 'relu'.
#### 3. Then finally we'll add a 37 neutron dense layer with 'softmax' as activation for our final prediction.

In [50]:
model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss="categorical_crossentropy",
    metrics=[keras.metrics.CategoricalAccuracy()],
)

epochs = 50
print("Fitting the top layer of the model")
model.fit(train_generator, epochs=epochs, validation_data=val_generator)

Fitting the top layer of the model
Epoch 1/50
184/184 [==============================] - 1022s 6s/step - loss: 3.8186 - categorical_accuracy: 0.0409 - val_loss: 3.2547 - val_categorical_accuracy: 0.1238
Epoch 2/50
184/184 [==============================] - 994s 5s/step - loss: 3.3424 - categorical_accuracy: 0.0870 - val_loss: 2.4456 - val_categorical_accuracy: 0.2694
Epoch 3/50
184/184 [==============================] - 993s 5s/step - loss: 2.9532 - categorical_accuracy: 0.1515 - val_loss: 2.0001 - val_categorical_accuracy: 0.3497
Epoch 4/50
184/184 [==============================] - 992s 5s/step - loss: 2.6839 - categorical_accuracy: 0.2082 - val_loss: 1.6755 - val_categorical_accuracy: 0.4626
Epoch 5/50
184/184 [==============================] - 992s 5s/step - loss: 2.5475 - categorical_accuracy: 0.2420 - val_loss: 1.5729 - val_categorical_accuracy: 0.4680
Epoch 6/50
184/184 [==============================] - 992s 5s/step - loss: 2.4003 - categorical_accuracy: 0.2810 - val_loss: 1.44

### We'll adjust the extended layer first with 50 epochs. The training starts with cross entropy loss of 3.2547, categorical accuracy 0.1238 and ends with val_cross_entropy_loss: 0.9268 - val_categorical_accuracy: 0.6966 which is a great improvement.  

In [52]:
model.save("C:\\Users\\Dell\\Documents\\projects\\image search engine\\img_model1.keras")

In [8]:
#### Saving the intermediate model

In [53]:
base_model.trainable = True
model.summary(show_trainable=True)

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss="categorical_crossentropy",
    metrics=[keras.metrics.CategoricalAccuracy()],
)

epochs = 50
print("Fitting the end-to-end model")
model.fit(train_generator, epochs=epochs, validation_data=val_generator)

Model: "model_5"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_16 (InputLayer)       [(None, 224, 224, 3)]     0         Y          
                                                                            
 vgg16 (Functional)          (None, 7, 7, 512)         1471468   Y          
                                                       8                    
                                                                            
 global_average_pooling2d_8  (None, 512)               0         Y          
  (GlobalAveragePooling2D)                                                  
                                                                            
 dropout_11 (Dropout)        (None, 512)               0         Y          
                                                                            
 dense_11 (Dense)            (None, 180)               9234

### Now we'll adjust the base model weights with very low learning rate as we don't want alter the weights it got from imagenet too much. we started with val_loss: 0.8620 - val_categorical_accuracy: 0.7102 and ended with val_loss: 0.6658 - val_categorical_accuracy: 0.7986 wich is great considering we don't have the sufficient hardware for complex processing ability.  
### we did try multiple combination of layers with different number of neutrons using some domain knowledge. But bcoz of huge training time we were unable to perform hyper parameter tuning.

In [54]:
print("Test dataset evaluation")
model.evaluate(test_generator)

Test dataset evaluation
733/733 [==============================] - 109s 149ms/step - loss: 0.6603 - categorical_accuracy: 0.8104


[0.6603395342826843, 0.8103683590888977]

### Test results

In [55]:
model.save("C:\\Users\\Dell\\Documents\\projects\\image search engine\\img_model360_1final.keras")

#### Saving the final model

In [75]:
predictions=model.predict(test_generator)

733/733 [==============================] - 119s 162ms/step


In [77]:
y_pred = np.rint(predictions)

In [83]:
print(y_pred[])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [37]:
print(x)

KerasTensor(type_spec=TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), name='dropout_1/Identity:0', description="created by layer 'dropout_1'")


In [21]:
#some random testing


img=Image.open("C:\\Users\\Dell\\Documents\\projects\\image search engine\\oxford-iiit-pet\\images.tar\\images\\images\\american_pit_bull_terrier_97.jpg")
#img=img.resize(224,224)
img = img.resize((224, 224))
# Convert the image color space
img = img.convert('RGB')
# # # Reformat the image
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)
fea=base_model.predict(img)
x = keras.layers.GlobalAveragePooling2D()(fea)
x = keras.layers.Dropout(0.2)(x)  
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary(show_trainable=True)

1/1 [==============================] - 0s 254ms/step


In [26]:
fea.shape

(1, 7, 7, 512)

In [23]:
x

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[2.6796570e+00, 0.0000000e+00, 5.0214291e+00, 2.2768803e+00,
        5.9144697e+00, 1.3501382e+00, 2.1654837e+00, 3.6408064e-01,
        3.2541311e+00, 2.5748460e+00, 1.1873509e+01, 1.2121794e+00,
        6.7951880e+00, 5.1907516e+00, 2.8853548e+00, 5.2441449e+00,
        1.6723387e+00, 2.6988816e-01, 2.0325974e-02, 9.2371470e-01,
        8.0372256e-01, 8.8719529e-01, 9.3356621e-01, 4.1201520e+00,
        6.1037936e+00, 1.2726702e-01, 1.1162183e+00, 7.3147243e-01,
        2.4614964e+01, 2.9078398e+00, 1.4383894e+00, 9.7435838e-01,
        2.1254482e+00, 4.4116569e+00, 7.4129815e+00, 5.6852090e-01,
        1.1219274e+01, 4.4416779e-01, 3.3469326e+00, 1.2927494e+00,
        1.1640214e+01, 1.0171223e+01, 1.0356932e+00, 2.3694719e-01,
        3.8187263e+00, 2.2734046e+00, 1.2757719e+01, 1.8919638e+00,
        7.4232256e-01, 1.7191716e+00, 1.9168786e+00, 3.2091103e-02,
        7.1641893e+00, 5.3928103e+00, 8.7730426e-01, 2.9905784e+00

In [27]:
x.shape

TensorShape([1, 512])

In [28]:
#testing end

### Example (testing with a sample image)

In [ ]:
#1
model1=keras.models.load_model("C:\\Users\\Dell\\Documents\\projects\\image search engine\\img_model360_1final.keras")

In [ ]:
#2
model1=keras.saving.load_model("C:\\Users\\Dell\\Documents\\projects\\image search engine\\img_model360_1final.keras")

In [313]:
img=Image.open("C:\\Users\\Dell\\Documents\\projects\\image search engine\\oxford-iiit-pet\\images.tar\\images\\images\\american_bulldog_78.jpg")
img = img.resize((224, 224))
img = img.convert('RGB')
# # # Reformat the image
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)
a=model1.predict(img)

1/1 [==============================] - 0s 136ms/step


In [314]:
a

array([[4.1175182e-15, 9.5238996e-01, 4.8162153e-08, 1.1638653e-15,
        7.9612373e-06, 4.1725457e-06, 7.8631608e-08, 4.6858513e-06,
        7.5372841e-13, 1.4013742e-08, 1.5929876e-17, 1.7887800e-10,
        4.8810891e-03, 1.2372130e-16, 9.8154984e-10, 2.1956817e-16,
        4.3476377e-23, 7.5757061e-14, 5.2479369e-04, 1.8927030e-15,
        1.6793585e-14, 3.1410184e-02, 2.0921407e-11, 4.0804065e-04,
        4.7115127e-14, 2.0337441e-06, 5.2410619e-12, 4.5276316e-10,
        5.1428471e-03, 1.8079931e-09, 7.3156143e-15, 1.1190661e-03,
        1.4038112e-12, 1.6714974e-14, 1.0522134e-18, 4.1050469e-03,
        5.2652635e-14]], dtype=float32)

In [315]:
p=np.rint(a)
p=list(p)
p

[array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.], dtype=float32)]

In [316]:
classes={'abyssinian':[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'american_bulldog':[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'american_pitbull_terrier':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'basset_hound':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'beagle':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0.]],
        'bengal':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0.]],
        'birman':[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0.]],
        'bombay':[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0.]],
        'boxer':[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0.]],
        'british_short_hair':[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1.]],
        'chihuaua':
         [[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'egyptian_mau':              
         [[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'english_cocker_spaniel':
         [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'english_setter':
         [[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
         'german_shorthair':
         [[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
         'great_pyrenees':
         [[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'havanese':
         [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        
        'japanese_chin':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'keeshond':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'leonberger':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'maine_coon':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'miniature_pinsch':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'newfoundland':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'persian':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'pomerenian':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'pug':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'ragdoll':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'russian_blue':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'saint_bernard':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'samoyed':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'scottish_terrier':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'shiba_inu':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'siamese':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'sphynx':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'staffordshire_bullterrier':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0.]],
        'wheten_terrier':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0.]],
        'yorkshire_terrier':
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0.]]}
        
       

In [317]:
ind=list(p[0]).index(1)
for k in classes:
        ind=list(p[0]).index(1)
        val=list(classes[k][0]).index(1)
        if val==ind:
            print(k)
            break

american_bulldog


#### The model is working fine